In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import os
import pymongo
from splinter import Browser
import re
import time

In [2]:
#for windows users for splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [3]:
# NASA Mars News 
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(5)

html = browser.html
soup = bs(html, 'html.parser')

news_title_results = soup.find_all('div', class_='content_title')[1].text
news_body_results = soup.find_all('div', class_='article_teaser_body')[0].text
print(f"Latest news title: {news_title_results}")
print(f"Latest news text: {news_body_results}")


Latest news title: NASA's Mars Rover Drivers Need Your Help
Latest news text: Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.


# JPL Mars Space Images - Featured Image

In [4]:
# JPL Mars Space Images - Featured Image

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(5)

In [5]:
# find url for featured image

image_url = browser.find_by_css('article')['style'].replace('background-image: url("', '').replace('");', '')
main_url = 'https://www.jpl.nasa.gov'
featured_image_url = main_url + image_url
print(f"Featured image url: {featured_image_url}")

Featured image url: https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14872-1920x1200.jpg


# Mars Weather

In [6]:
# Mars Weather
# URL of page to be scraped
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
time.sleep(10)

In [7]:
weather_html = browser.html
weather_soup = bs(weather_html, 'html.parser')

mars_weather = weather_soup.find(attrs={"data-testid": "tweet"})
weather_text = mars_weather.text
mars_weather_list = weather_text.split("InSight")
mars_weather_today = (mars_weather_list[1])
print(f"Latest Mars Weather Report: {mars_weather_today}")

Latest Mars Weather Report:  sol 547 (2020-06-10) low -92.2ºC (-133.9ºF) high -1.0ºC (30.1ºF)
winds from the SW at 4.5 m/s (10.0 mph) gusting to 14.2 m/s (31.9 mph)
pressure at 7.40 hPa11026


# Mars Facts Table

In [8]:
## Mars Facts Table
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
time.sleep(10)
facts_html = browser.html
facts_soup = bs(facts_html, 'html.parser')

In [9]:
# read facts table from url
facts_table = pd.read_html(facts_url)
facts_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [10]:
# convert html table into dataframe
facts_df = facts_table[0]
facts_df.columns = ['Description', 'Value']
facts_df.set_index('Description', inplace=True)
facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
# convert dataframe into html table
mars_html_table = facts_df.to_html()
mars_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [12]:
# create separate html file for facts table
mars_html_table.replace('\n', '')
facts_df.to_html('./templates/mars_fact_table.html')

# Mars Hemispheres

In [13]:
#Mars Hemispheres
#url for mars hemispheres
mars_hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemispheres_url)
time.sleep(5)
mars_hemisphere_html = browser.html
hemisphere_soup = bs(mars_hemisphere_html, 'html.parser')

In [14]:
#create empty dictionary for hemisphere info 
hemispheres = hemisphere_soup.find_all('div', class_='item')
mars_hemisphere_list = []


#run thru the url for each hemisphere
for each in hemispheres:

    #find hemisphere name & link to image file
    hemi_name = each.find('h3').text
    split = hemi_name.split('Enhanced')
    hemisphere_name = split[0]
    browser.click_link_by_partial_text(hemi_name)
    hemi_page_html = browser.html
    hemi_soup = bs(hemi_page_html, "html.parser")
    download = hemi_soup.find('div', class_="downloads")
    hemi_url = download.a["href"]
    hemisphere_dict = {"name": hemisphere_name, "image_url": hemi_url}
    mars_hemisphere_list.append(hemisphere_dict)
    browser.back()
    time.sleep(3)
    
print(mars_hemisphere_list)    

C:\Anaconda\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


[{'name': 'Cerberus Hemisphere ', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'name': 'Schiaparelli Hemisphere ', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'name': 'Syrtis Major Hemisphere ', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'name': 'Valles Marineris Hemisphere ', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
